
# **Lab #2. Outlier and Imputation**

In this lab, we are going to learn how to (1) detect ourliers, and (2) handle missing values.

*This material is a joint work of TAs from IC Lab at KAIST, including Woohyeok Choi, Yunjo Han, Soowon Kang, Auk Kim, Inyeob Kim, Minhyung Kim, Hansoo Lee, Cheul Y. Park, Eunji Park, Sangjun Park, and Panyu Zhang. This work is licensed under CC BY-SA 4.0.*

## Preparation

We will proceed with installing Python libraries to be used and mounting the drive in Colab.

In [62]:
!pip install plotly pandas scipy scikit-learn requests

In [63]:
import os, sys
from google.colab import drive

mount_dir = '/content/drive'
drive.mount(mount_dir) # Mount Google Drive

class_dir = os.path.join(mount_dir, "My Drive", "IoT_Data_Science")
sys.path.insert(0, class_dir) # add class folder to sys.path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


First, please download magnetometer_phone dataset file (about  48MB) and ppg dataset file (about 236KB) by running the code below.

In [64]:
import requests

url_dict = {
    os.path.join("crowdsignal","MagnetometerPhone.csv"): "https://docs.google.com/uc?export=download&id=1tkUpCQxn3bmxqS6OnawpBk-jJXxRFTSf",
    os.path.join("supplementary", "PPG.csv"): "https://docs.google.com/uc?export=download&id=1p-0ACqKqCnHT2tPuTGmFntHOxMSlFzxG",
}

if not os.path.exists(os.path.join(class_dir,"crowdsignal")):
  os.mkdir(os.path.join(class_dir,"crowdsignal"))

if not os.path.exists(os.path.join(class_dir,"supplementary")):
  os.mkdir(os.path.join(class_dir,"supplementary"))

for data_path, url in url_dict.items():
    response = requests.get(url)
    open(os.path.join(class_dir, data_path),"wb").write(response.content)

The below method returns magnetometer_phone data. Please note that due to the size of data, we will use only the first 500 rows (out of 176,833 rows).

In [65]:
import os
import pandas as pd

def get_simple_magnetometer_phone(size = 500):
  result = pd.read_csv(os.path.join(class_dir, 'crowdsignal', 'MagnetometerPhone.csv')).sort_values(by=['timestamp']).head(size)
  result.drop_duplicates(inplace=True)
  result.reset_index(inplace=True, drop=True)
  result['readableTimestamp'] = pd.to_datetime(result['timestamp'], unit='ns')
  return result

## Outlier Detection

###  Distribution-Based Models

A very simple approach of outlier detection is removing the spuriously high or low value.<br/>
Assuming the dataset follows a normal distribution, we can identify outliers using the following methods:

* IQR rule (also the same as how boxplot calculates outliers): [Q1 - 1.5IQR, Q3 + 1.5IQR]
    * Q1: The first Quartile
    * Q3: The third Quartile
    * IQR: Inter Quartile Range, Q3-Q1    
* 3MAD rule: $[\text{Median} - 3 \text{MAD}, \text{Median} + 3 \text{MAD}]$
    * Median: The second Quartile
    * MAD: Median Absolute Deviation, $\text{median}(|X_i - median(X)|)$
* $3\sigma$ rule: $[\mu - 3\sigma, \mu + 3\sigma]$
    * $\mu$: Mean
    * $\sigma$: Standard Deviation
* Chauvenet's Criterion: $[\mu - Z^{-1}_{\frac{1}{4n}}\sigma , \mu + Z^{-1}_{\frac{1}{4n}}\sigma]$
    * $Z^{-1}_{\frac{1}{4n}}$: Z-score covering the area that corresponds to the probability of $\frac{1}{4n}$  

#### Normality test

As mentioned, to apply Chauvenet's criterion, data should follow a normal distribution. We are going to check whether crowdsignal data follow a normal distribution.
Using `scipy.stats.normaltest()`, we can test normality of input data.

In [66]:
from scipy import stats

def isNormallyDistributed(data, sig = 0.05):
  # A low p-value indicates that the data may not follow a normal distribution.
  # In this function we used .05 as a default for threshold.
  _, p = stats.normaltest(data)
  return p >= sig

In [67]:
import numpy as np
x = np.random.normal(0,1,100) # generate an array that has 100 random samples from a uniform distribution over [0, 1).
display(x)
print('M =%f, SD = %f'%(np.mean(x), np.std(x)))
display(isNormallyDistributed(x))

array([-0.7307414 , -1.27991973, -0.23739196,  1.87200568, -0.01944214,
       -0.93259166, -1.29494247, -0.1051185 , -1.99496157, -1.30019284,
        1.54306765,  1.32326381, -0.12841215, -0.46566477,  0.0585089 ,
        2.34799693,  0.40497551,  0.90973817, -0.20245753,  0.06089661,
        0.6471627 ,  0.12609483, -1.89737698,  1.28054076, -0.61513368,
        2.11131764,  0.68684851,  0.07097242,  0.43932714, -0.10849789,
       -0.48643277,  0.20759826,  0.88499047, -0.46452975, -0.62223716,
       -0.35548864,  0.95201109,  1.45255025,  0.36201391,  0.58131413,
       -0.62970086,  0.87675465,  0.55173701, -0.71012796, -1.78767161,
       -1.2205701 , -0.66920273,  1.16193732,  1.55285978, -0.43951788,
       -0.40791535,  0.26162306,  0.01214651,  0.93093408, -0.52318809,
        0.94914299, -0.2392248 ,  0.85397471, -0.27104046, -0.61309468,
        1.91231764,  1.06522708, -1.7920451 , -0.44677351,  1.20492292,
       -1.19666047,  0.77067833,  0.6670561 , -0.69914658, -0.00

M =0.109171, SD = 0.983371


True

In [68]:
# generate an array that has uniform distribution
x = np.random.uniform(size = 100)
display(x)
print('M =%f, SD = %f'%(np.mean(x), np.std(x)))
display(isNormallyDistributed(x))

array([0.9929648 , 0.07379656, 0.55385428, 0.96930254, 0.52309784,
       0.62939864, 0.69574869, 0.45454106, 0.62755808, 0.58431431,
       0.90115801, 0.04544638, 0.28096319, 0.95041148, 0.89026378,
       0.45565675, 0.6201326 , 0.27738118, 0.18812116, 0.4636984 ,
       0.35335223, 0.58365611, 0.07773464, 0.97439481, 0.98621074,
       0.69816171, 0.53609637, 0.30952762, 0.81379502, 0.68473117,
       0.16261694, 0.91092718, 0.82253724, 0.94979991, 0.72571951,
       0.6134152 , 0.41824304, 0.93272848, 0.86606389, 0.04521867,
       0.02636697, 0.37646337, 0.81055333, 0.98727613, 0.15041689,
       0.59413072, 0.38089086, 0.9699144 , 0.84211892, 0.8383287 ,
       0.46869316, 0.4148195 , 0.27340707, 0.0563755 , 0.86472238,
       0.81290101, 0.99971767, 0.99663684, 0.55543171, 0.76898742,
       0.94476573, 0.84964739, 0.2473481 , 0.45054414, 0.12915942,
       0.95405103, 0.60617463, 0.22864281, 0.67170068, 0.61812824,
       0.35816272, 0.11355759, 0.6715732 , 0.5203077 , 0.77231

M =0.565398, SD = 0.288845


False

Let's do a normality test for magnetometer data.

In [69]:
simple_magnetometer_phone = get_simple_magnetometer_phone(500) # Get first 500 rows from magnetometer_phone data
x = simple_magnetometer_phone["x"]
print('M =%f, SD = %f'%(np.mean(x), np.std(x)))
display(isNormallyDistributed(x))

M =-76.510680, SD = 0.822605


False

It turned out it's not normally distributed. In this case, we need to visually inspect the distribution to see how the distribution looks like.

In [70]:
import plotly.express as px

fig = px.histogram(simple_magnetometer_phone, x="x")
fig.show()

#### IQR

The IQR of a set of values is calculated as the difference between the upper and lower quartiles, Q3 and Q1. Each quartile is a median calculated as follows.
*   First quartile Q1 = the value under which 25% of data points are found when they are arranged in increasing order.
*   Third quartile Q3 = the value under which 75% of data points are found when arranged in increasing order.
*   Second quartile Q2 is the same as the ordinary median

Through IQR, we can detect the outlier. The value obtained by subtracting 1.5 * IQR from Q1 is set as the minimum limit line, and the value obtained by adding 1.5 * IQR from Q3 is set as the maximum limit line.

<img src="https://drive.google.com/uc?id=17HH_wo-2w7z6Ebrjfkzf94R55nrqIG8W" width="500px" height="250px" title="IQR"></img>

Let's calculate IQR.

And get the value of lower threshold and upper threshold by substracting 1.5 * IQR from Q1 and adding 1.5 * IQR from Q3.

In [71]:
q1=simple_magnetometer_phone['x'].quantile(0.25)
q3=simple_magnetometer_phone['x'].quantile(0.75)
iqr=q3-q1

lower_threshold = q1-1.5*iqr
print(lower_threshold)
upper_threshold = q3+1.5*iqr
print(upper_threshold)

-77.46000000000001
-76.02000000000001


So, we discard the outside scope of -77.46 ~ -76.02 as the outlier.

In [72]:
outlier_index = simple_magnetometer_phone.query("x < @lower_threshold or x > @upper_threshold").index
simple_magnetometer_phone['x_outlier'] = simple_magnetometer_phone.index.isin(outlier_index)

In [73]:
import plotly.express as px
import pandas as pd

fig = px.histogram(simple_magnetometer_phone, x="x",
                   color="x_outlier", color_discrete_map={False:'#636efa', True:'#ef553b'}) # showing different color depending on boolean value (TRUE or FALSE) in x_outliner column
fig.show()

#### MAD

Let's get the median and median absolute deviation of data to calculate the lower threshold and upper threshold.

In [74]:
x = simple_magnetometer_phone['x'].values
median = np.median(x)
mad= np.median(np.absolute(x - np.median(x)))

lower_threshold = median-3*mad
print(lower_threshold)
upper_threshold = median+3*mad
print(upper_threshold)

-77.21999999999998
-76.14000000000003


In [75]:
outlier_index = simple_magnetometer_phone.query("x < @lower_threshold or x > @upper_threshold").index
simple_magnetometer_phone['x_outlier'] = simple_magnetometer_phone.index.isin(outlier_index)

In [76]:
import plotly.express as px
import pandas as pd

fig = px.histogram(simple_magnetometer_phone, x="x",
                   color="x_outlier", color_discrete_map={False:'#636efa', True:'#ef553b'}) # showing different color depending on boolean value (TRUE or FALSE) in x_outliner column
fig.show()

#### Chauvenet’s criterion

Below chauvenet method returns a list of outliers (True = outlier). <br/>
Although our dataset doesn't follow normal distribution, but let's simply apply this technique just for the sake of testing.

In [77]:
import scipy

def chauvenet(df):
    mean = df.mean()              # Mean of incoming array
    std = df.std()                # Standard deviation
    N = len(df)                   # Lenght of incoming array
    criterion = 1.0/(4*N)         # Chauvenet's criterion
    d = abs(df-mean)/std          # Distance of a value to mean in stdv's
    d /= 2.0**0.5                 # The left and right tail threshold values
    prob = scipy.special.erfc(d)  # Area normal dist. - erfc: Complementary error function, 1 - erf(x).
                                  # https://www.johndcook.com/erf_and_normal_cdf.pdf
    return prob <= criterion      # Return Boolean array that indicates outlier


chauvenet(simple_magnetometer_phone.x)

,x
0,True
1,True
2,True
3,True
4,True
...,...
495,False
496,False
497,False
498,False


In [78]:
simple_magnetometer_phone['x_outlier'] = chauvenet(simple_magnetometer_phone["x"])
print('number of x_outlier: {}'.format(simple_magnetometer_phone.query("x_outlier == True").shape[0]))

number of x_outlier: 17


In [79]:
import plotly.express as px
import pandas as pd

fig = px.histogram(simple_magnetometer_phone, x="x",
                   color="x_outlier", color_discrete_map={False:'#636efa', True:'#ef553b'}) # showing different color depending on boolean value (TRUE or FALSE) in x_outliner column
fig.show()

### Mini-Exercise #1
To identify outliers in the x-axis of the magnetometer, please use the first 600 rows of crowdsignal data and Chauvenet's criterion.

Additionally, please create a line graph using timestamps and x-values, and represent outliers using color.

(Hint: timestamp is recorded in nano seconds!)

Graph will look like below:

<img src="https://drive.google.com/uc?id=1-4-UdOUNCYzBOAjG4_O6gyVcLikCLTac" width="1000px" height="300px" title="Q2"></img>

In [80]:
# Write your code here
fig = px.line(simple_magnetometer_phone, x="readableTimestamp", y="x", color="x_outlier", color_discrete_map={False:'#636efa', True:'#ef553b'})
fig.show()

### Distance-Based Models

Another way for detecting outliers is to calculate the distance from other samples.

* Simple Distance Method: If a percentage of samples greater than `min_frequency` have a distance greater than `min_distance`, sample will be considered as an outlier.
* Local Outlier Factor (LOF): LOF is a metric that shows how much lower the density of a point is compared to those of its neighboring points, calculated through its distance to its neighbors. If the LOF value is significantly greater than 1, it is considered an outlier.

#### Simple Distance Method

In [81]:
import scipy

def get_normalized(df):
  return (df - df.min()) / (df.max() - df.min())

def simple_distance_based_outlier_detection(df, min_distance, min_frequency):
  df_norm = df.copy()
  # Normalize data
  df_norm = get_normalized(df_norm)

  # Calculate pairwise distances of all rows.
  pairwise_distance = scipy.spatial.distance.pdist(df_norm, 'euclidean')
  distance_matrix = scipy.spatial.distance.squareform(pairwise_distance)
  distance_df = pd.DataFrame(distance_matrix, columns=df_norm.index, index = df_norm.index)

  # Calculate number of sample that is far than min_distance
  df_greater_than_min_dis = distance_df[distance_df > min_distance].count()

  # Calculate whether percentage of sample is greater than min_frequency or not.
  df_greater_than_min_frequency = df_greater_than_min_dis/df.shape[0] > min_frequency

  df['outlier'] = df_greater_than_min_frequency
  return df

Now, using `simple_distance_based_outlier_detection()`, we can detect outliers using a distance-based model.

We can apply the function to magenetomer's x and y-axis.

In [82]:
import plotly.graph_objects as go
import pandas as pd

fig = go.Figure()

simple_magnetometer_phone = get_simple_magnetometer_phone()[['x','y']]
simple_magnetometer_phone = get_normalized(simple_magnetometer_phone)

min_distance = 0.5
min_frequency = 0.5
simple_magnetometer_phone = simple_distance_based_outlier_detection(simple_magnetometer_phone, min_distance, min_frequency)

fig = px.scatter(simple_magnetometer_phone, x = "x", y= "y", color = "outlier", color_discrete_map={False:'#636efa', True:'#ef553b'} )
fig.update_layout(width = 500, height = 500)
fig.show()

### Mini-Exercise #2

Please try how to set `min_distance` and `min_freqeuncy` to remove outliers as shown in the figure below.

<img src="https://drive.google.com/uc?id=1jWv-t1mhYlbSaeg973B4B5bSfgvqRBq9" width="500px" height="500px" title="Q2"></img>

In [83]:
# Write your code here
import plotly.graph_objects as go
import pandas as pd

fig = go.Figure()

simple_magnetometer_phone = get_simple_magnetometer_phone()[['x','y']]
simple_magnetometer_phone = get_normalized(simple_magnetometer_phone)

min_distance = 0.2
min_frequency = 0.3
simple_magnetometer_phone = simple_distance_based_outlier_detection(simple_magnetometer_phone, min_distance, min_frequency)

fig = px.scatter(simple_magnetometer_phone, x = "x", y= "y", color = "outlier", color_discrete_map={False:'#636efa', True:'#ef553b'} )
fig.update_layout(width = 500, height = 500)
fig.show()

#### Local Outlier Factors

Instead of implementing LocalOutlierFactors, scikit-learn provides LocalOutlierFactor function as `sklearn.neighbors.LocalOutlierFactor()`.

Let's learn usage of the fuction.

In [84]:
import numpy as np
import plotly.graph_objects as go
from sklearn.neighbors import LocalOutlierFactor

np.random.seed(42)

# Generate train data
# Make two clusters nearby (2,2) and (-2,-2)
cluster = 0.3 * np.random.randn(100, 2)
samples = np.concatenate((cluster + 2, cluster-2), axis = 0)

# Make 20 outliers
outliers = np.random.uniform(low=-4, high=4, size=(20, 2))
samples = np.concatenate((samples, outliers), axis = 0)

# generate model by selecting parameters
# n_neighbors: number of neighbor to consider while calculating LOF
# contamination: percentage of outlier considered as outlier
clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)

# use fit_predict to compute the predicted labels of the training samples
is_outlier = clf.fit_predict(samples)
LOF = clf.negative_outlier_factor_

We can visualize outlier scores of each data point.

In [85]:
fig = go.Figure()
# Draw samples
fig.add_trace(go.Scatter(x=samples[:,0], y=samples[:,1], name='Data points', mode='markers', marker=dict(size=2, color='black')))
# Draw LOF as a radius
radius = (LOF.max() - LOF) / (LOF.max() - LOF.min())
fig.add_trace(go.Scatter(x=samples[:,0], y=samples[:,1], name='LOF scores',  mode='markers', marker=dict(opacity=0.25, size=radius*100)))

fig.update_layout(
    title="Local Outlier Factor (LOF)",
    width=500,
    height=500,
    xaxis = dict(range = [-5,5]),
    yaxis = dict(range = [-5,5]),
    legend=dict(yanchor='top', y=0.99, xanchor='left', x=0.01))
fig.show()

Also, we can check which data points are outlier.

In [86]:
is_outlier = list(map(lambda x: 'outlier' if x == -1 else 'normal', is_outlier))

fig = px.scatter(x=samples[:,0], y=samples[:,1], color = is_outlier, color_discrete_map={'normal':'#636efa', 'outlier':'#ef553b'})
fig.update_layout(
    title="Local Outlier Factor (LOF)",
    width=500,
    height=500,
    xaxis = dict(range = [-5,5]),
    yaxis = dict(range = [-5,5]),
    legend=dict(yanchor='top', y=0.99, xanchor='left', x=0.01))
fig.show()

Let's apply the LOF to our magenetomer data

In [87]:
data = get_normalized(get_simple_magnetometer_phone()[['x','y']])

clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)

is_outlier = clf.fit_predict(data)
LOF = clf.negative_outlier_factor_

In [88]:
fig = go.Figure()
# Draw samples
fig.add_trace(go.Scatter(x=data['x'], y=data['y'], name='Data points', mode='markers', marker=dict(size=2, color='black')))
# Draw LOF as a radius
radius = (LOF.max() - LOF) / (LOF.max() - LOF.min())
fig.add_trace(go.Scatter(x=data['x'], y=data['y'], name='LOF scores',  mode='markers', marker=dict(opacity=0.25, size=radius*100)))

fig.update_layout(
    title="Local Outlier Factor (LOF)",
    width=500,
    height=500,
    xaxis = dict(range = [0,1]),
    yaxis = dict(range = [0,1]),
    legend=dict(yanchor='top', y=0.99, xanchor='left', x=0.01))
fig.show()

It showed that it is hard to capture outlier using LOF for our data.

In [89]:
is_outlier = list(map(lambda x: 'outlier' if x == -1 else 'normal', is_outlier))
fig = px.scatter(x=data['x'], y=data['y'], color = is_outlier, color_discrete_map={'normal':'#636efa', 'outlier':'#ef553b'})

fig.update_layout(
    title="Local Outlier Factor (LOF)",
    width=500,
    height=500,
    xaxis = dict(range = [0,1]),
    yaxis = dict(range = [0,1]),
    legend=dict(yanchor='top', y=0.99, xanchor='left', x=0.01))
fig.show()

## Imputation of Missing Values

Using [*pandas.DataFrame.fillna*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html), we can fill *NA/NaN* (Not Available; Not a Number) values using the specified method.

In [90]:
import pandas as pd

# Impute the mean values in case if missing data.
def impute_mean(dataset, col):
    dataset[col] = dataset[col].fillna(dataset[col].mean())
    return dataset

# Impute the median values in case if missing data.
def impute_median(dataset, col):
    dataset[col] = dataset[col].fillna(dataset[col].median())
    return dataset

Also, using  [*pandas.DataFrame.fillna*](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html), we can fill *NA/NAN* values using an interpolation method.

In [91]:
# Interpolate the dataset based on previous/next values..
def impute_interpolate(dataset, col):
    dataset[col] = dataset[col].interpolate()
    # And fill the initial data points if needed:
    dataset[col] = dataset[col].fillna(method='bfill')
    return dataset

In [92]:
# let's generate a 5x5 matrix where each cell is filled w/ a random number
def array_with_missing_value():
  x = np.random.rand(5,5)
  # we intentionally replace a cell (2, 2) with NaN
  x[2][2] = np.NaN
  return pd.DataFrame(data=x, columns=list('ABCDE'))

df = array_with_missing_value()
print('Original Dataframe')
display(df)
impute_mean(df, 'C')
print('---------------------------------------------------')
print('Imputed Dataframe')
display(df)

Original Dataframe


,A,B,C,D,E
0,0.726091,0.975852,0.516300,0.322956,0.795186
1,0.270832,0.438971,0.078456,0.025351,0.962648
2,0.835980,0.695974,NaN,0.173294,0.156437
3,0.250243,0.549227,0.714596,0.660197,0.279934
4,0.954865,0.737897,0.554354,0.611721,0.419600


---------------------------------------------------
Imputed Dataframe


,A,B,C,D,E
0,0.726091,0.975852,0.516300,0.322956,0.795186
1,0.270832,0.438971,0.078456,0.025351,0.962648
2,0.835980,0.695974,0.465927,0.173294,0.156437
3,0.250243,0.549227,0.714596,0.660197,0.279934
4,0.954865,0.737897,0.554354,0.611721,0.419600


For another library related to missing value imputation, please refer to [sklearn imputation of missing values](https://scikit-learn.org/stable/modules/impute.html).

### Mini-Exercise #3
Try imputation for numeric columns in *KEmoPhone_300L* data.

Please use mean and interpolation to impute missing values for all numeric columns and save the results as separate CSV files to output_dir.(KEmoPhone_300L_imputation_mean.csv and KEmoPhone_300L_imputation_interpolate.csv)

In [95]:
# Write your code here
KEmoPhone_300L = pd.read_csv(os.path.join(class_dir, 'output', 'KEmoPhone_300L.csv'))
KEmoPhone_300L.drop_duplicates( inplace = True)
KEmoPhone_300L.sort_values(by="timestamp", inplace = True)
KEmoPhone_300L.set_index("timestamp", inplace = True)

# impute missing values with mean
KEmoPhone_300L_mean_imputed = impute_mean(KEmoPhone_300L.copy(), col=["accX","accY", "accZ", "bpm"] )

# impute missing values with interpolation
KEmoPhone_300L_interpolate_imputed = impute_interpolate(KEmoPhone_300L.copy(), col=["accX","accY", "accZ", "bpm"] )

KEmoPhone_300L_mean_imputed.to_csv(os.path.join(class_dir, 'output', 'KEmoPhone_300L_imputation_mean.csv'))
KEmoPhone_300L_interpolate_imputed.to_csv(os.path.join(class_dir, 'output', 'KEmoPhone_300L_imputation_interpolate.csv'))

display(KEmoPhone_300L_mean_imputed.head(),KEmoPhone_300L_interpolate_imputed.head())

<ipython-input-91-87833c02e50a>:5: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



,readableTimestamp,accX,accY,accZ,bpm,motion
timestamp,,,,,,
1.557710e+12,2019-05-13 01:19:11.700,0.037354,0.532959,0.872070,73.949112,STILL
1.557710e+12,2019-05-13 01:19:12.000,0.035645,0.534546,0.871094,73.949112,STILL
1.557710e+12,2019-05-13 01:19:12.300,0.036377,0.535767,0.870972,78.000000,STILL
1.557710e+12,2019-05-13 01:19:12.600,0.038330,0.535767,0.873901,73.949112,STILL
1.557710e+12,2019-05-13 01:19:12.900,0.035645,0.531331,0.873372,73.949112,STILL


,readableTimestamp,accX,accY,accZ,bpm,motion
timestamp,,,,,,
1.557710e+12,2019-05-13 01:19:11.700,0.037354,0.532959,0.872070,78.0,STILL
1.557710e+12,2019-05-13 01:19:12.000,0.035645,0.534546,0.871094,78.0,STILL
1.557710e+12,2019-05-13 01:19:12.300,0.036377,0.535767,0.870972,78.0,STILL
1.557710e+12,2019-05-13 01:19:12.600,0.038330,0.535767,0.873901,78.0,STILL
1.557710e+12,2019-05-13 01:19:12.900,0.035645,0.531331,0.873372,78.0,STILL


## Practical Practice: Heartrate Data Processing (Outlier Handling and Data Normalization)

If sensor data are not clean and well edited, analysis will be flawed. So we need to do sensor data processing.

As the practical practice for data process, we will conduct outlier handling and normalization that are actually done in previous studies [1] for heart rate data filtering using K-EmoPhone heart rate data.

[1] Mishra, V., Sen, S., Chen, G., Hao, T., Rogers, J., Chen, C. H., & Kotz, D. (2020). Evaluating the reproducibility of physiological stress detection models. Proceedings of the ACM on Interactive, Mobile, Wearable and Ubiquitous Technologies, 4(4), 1-29. [PDF](https://digitalcommons.dartmouth.edu/cgi/viewcontent.cgi?article=5047&context=facoa)

1. We will load, clean and visualize data.

2. We will discard HR samples that are outside the range of heart-rate BPM based on the IQR outlier criteria, and the the $median\pm 3*MAD$ (median absolute deviation).

Load heartrate data

In [96]:
heartrate = pd.read_csv(os.path.join(class_dir, 'KEmoPhone', 'HeartRate.csv'))
heartrate.drop_duplicates( inplace = True)
heartrate.sort_values(by="timestamp", inplace = True)
heartrate.set_index("timestamp", inplace = True)
display(heartrate.head(10000))

,bpm,quality
timestamp,,
1557710352394,78,ACQUIRING
1557710353396,78,ACQUIRING
1557710354406,78,ACQUIRING
1557710355398,78,ACQUIRING
1557710356395,78,ACQUIRING
...,...,...
1557728731496,73,LOCKED
1557728732475,73,LOCKED
1557728733469,73,LOCKED


Heartrate data from Microsoft Band 2 has two types of quality singals which are ACQUIRING and LOCKED, and when the field is set ACQUIRING, it means that value is not valid. Let's remove the rows with the ACQUIRING state to ensure the quality heartrate data.

In [98]:
heartrate.query("quality != 'ACQUIRING'", inplace = True)

Then, create new timestamp column by using to_datetime. Also, drop NA values.

In [99]:
heartrate['readableTimestamp'] = pd.to_datetime(heartrate.index, unit='ms')
heartrate.dropna(axis=0, inplace = True)
heartrate.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20533 entries, 1557710369350 to 1557741126842
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   bpm                20533 non-null  int64         
 1   quality            20533 non-null  object        
 2   readableTimestamp  20533 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 641.7+ KB


Lastly, let's plot the cleaned BPM data.

In [100]:
fig = px.scatter(heartrate.head(10000), x="readableTimestamp", y="bpm")
fig.update_layout(
    yaxis = dict(range = [30,120])
)
fig.show()

Based on previous research conducted to find the maximum human heartrate [1,2], Mishra et al. [3,4] set the upper bound of heart rate to 220 bpm (based on some prior studies). Also, to determine the lower bound, we inspected heart-rate data of all the participants (visually) to find any noticeable value that would seem invalid.

The resulting range 30 ~ 220 bpm is fairly conservative, but we are confident that any data point outside this range is considered invalid (also, affirmed in some existing studies and based on the face validity criteria).

*   [1] S. Fox and W. Haskell. The exercise stress test: needs for standardization. Cardiology: Current Topics and Progress, New York Academic Press, pages 149–154, 1970.
*   [2] H. Tanaka, K. D. Monahan, and D. R. Seals. Agepredicted maximal heart rate revisited. Journal of the American College of Cardiology, 37(1):153–6, Jan 2001. Online at http://www.ncbi.nlm.nih.gov/pubmed/11153730.
*  [3] Mishra, V., Pope, G., Lord, S., Lewia, S., Lowens, B., Caine, K., ... & Kotz, D. (2018, October). The case for a commodity hardware solution for stress detection. In Proceedings of the 2018 ACM international joint conference and 2018 international symposium on pervasive and ubiquitous computing and wearable computers (pp. 1717-1728).
*  [4] Mishra, V., Sen, S., Chen, G., Hao, T., Rogers, J., Chen, C. H., & Kotz, D. (2020). Evaluating the reproducibility of physiological stress detection models. Proceedings of the ACM on Interactive, Mobile, Wearable and Ubiquitous Technologies, 4(4), 1-29.

In [101]:
heartrate["is_outlier"] = heartrate.index.isin(heartrate.query("bpm > 220 or bpm < 30").index)

Luckly, the K-EmoPhone heart rate data's minimum BPM is 38 and maximum BPM is 112. All samples can pass through the outlier detection range (30--220 BPM).  

Of course, you can change the threshold as you wish (which requires a strong justification though).

For the sake of illustration, let's handle the outliers using the criteria of IQR.

In [102]:
q1=heartrate['bpm'].quantile(0.25)
q3=heartrate['bpm'].quantile(0.75)
iqr=q3-q1

lower_threshold = q1-1.5*iqr
print(lower_threshold)
upper_threshold = q3+1.5*iqr
print(upper_threshold)

62.0
86.0


So, we discard the outside the scope of 62.0 BPM ~ 86.0 BPM range as the outlier.

In [103]:
heartrate["is_outlier"] = np.logical_or(heartrate["is_outlier"], heartrate.index.isin(heartrate.query("bpm > @upper_threshold or bpm < @lower_threshold").index))

In [104]:
fig = px.scatter(heartrate.head(10000), x="readableTimestamp", y="bpm", color = "is_outlier", color_discrete_map = {False:'#636efa', True:'#ef553b'})
fig.update_layout(
    yaxis = dict(range = [30,120])
)
fig.show()

# Homework #2: Preprocessing KAIST dataset

*   3 points toward your final score.
*   Homework #2 is due at **23:59:59 on 3/19 (Wednesday)**.
*   If you get memory error, please shorten the size of data (e.g., use first 1000 rows instead).

### Q1

Please (1) apply chauvenet outlier detection method for GSR data from K-EmoPhone dataset and (2) plot a graph with the outliers in blue, similar to what we did. **(0.75 pt)**

*   Apply chauvenet outlier detection method (0.5 pt)
*   Plot a histogram of GSR values with the outliers in red (0.25 pt)
*   You should show exactly the same graph as the one below. (e.g., axis, label, color, format) and if you don't show graph results, you won't get points

Number of Resistance_outlier: 1948

<img src="https://drive.google.com/uc?id=1ACwXUOLtMxIlhT287iKRAmDb6Xs15Qb8" title="Q1"></img>


In [105]:
# Write your code here
gsr = pd.read_csv(os.path.join(class_dir, 'KEmoPhone', 'GSR.csv'))
#gsr = gsr[:1000]
gsr['resistance_outlier'] = chauvenet(gsr['Resistance'])

#print(gsr.query("resistance_outlier == True").count())
print("Number of Resistance_outlier:", gsr["resistance_outlier"].sum())

fig = px.histogram(gsr, x='Resistance', color='resistance_outlier')
fig.show()

Number of Resistance_outlier: 1948


### Q2

Please remove outliers by applying simple distribution-based outlier detection method for Photoplethysmography (PPG) data (timestamp and PPG) from supplementary dataset. **(0.75 pt)**

*   Apply MAD detection method (0.25 pt)
*   Plot a graph after removing outliers - only data in normal range (0.5 pt)
*   You should show exactly the same graph as the one below. (e.g., axis, label, color, format) and if you don't show graph results, you won't get points

For more information regarding PPG, please refer to the [link](https://www.fibricheck.com/the-fibricheck-guide-to-analyse-ppg-measurements/#:~:text=The%20visualisation%20used%20for%20FibriCheck%20measurements&text=Graph%201%3A%20The%2060%2Dsecond,the%20heart%20rate%20is%2080.)

<img src="https://drive.google.com/uc?id=1mmxSoQTwDrAmurwOZ-BK8Fm91B0yFctf" title="Q2"/>

In [106]:
# Write your code here
#load data
ppg = pd.read_csv(os.path.join(class_dir,'supplementary',"PPG.csv"))

x = ppg['ppg']
median = np.median(x)
mad = np.median(np.absolute(x- np.median(x)))

lower_threshold = median- 3*mad
print("lower threshold:" , lower_threshold)

upper_threshold = median+ 3*mad
print("upper threshold:" , upper_threshold)

outlier_index = ppg.query("ppg < @lower_threshold or ppg > @upper_threshold").index
ppg.drop(outlier_index, inplace = True)
#ppg['outlier'] = ppg.index.isin(outlier_index)

# make timestamp readable
ppg['readableTimestamp'] = pd.to_datetime(ppg['timestamp'], unit='ms')

fig = px.line(ppg, x='readableTimestamp',y="ppg")
fig.show()

lower threshold: -1.2575292709202858e-41
upper threshold: 1.1579399688455758e-41


### Q3

Please remove the outlier data of GSR's resistance data with $median\pm3\text{MAD}$. **(0.75 points)**

**Only use first 1000 data**

*   Get the median, MAD, standard of lower threshold (median-3*MAD) and upper threshold (median+3*MAD) for outlier detection and show the values of median, mad, lower & upper threshold.(0.25 points)
*   Remove the outlier data. (0.25 points)
*   Plot the graph after removing the outlier data. (0.25 points)
*   You should show exactly the same graph as the one below. (e.g., axis, label, color, format) and if you don't show graph results, you won't get points


median: 5838.0<br/>
mad: 335.0<br/>
lower_threshold: 4833.0<br/>
upper_threshold: 6843.0

<img src="https://drive.google.com/uc?id=1-3QImRK48PLChI8Oyf9TFRVzYkikXgIs" title="Q3"></img>

In [107]:
# Write your code here
gsr = pd.read_csv(os.path.join(class_dir, 'KEmoPhone', 'GSR.csv'))
# only use the first 1000
gsr = gsr[:1000]

x = gsr["Resistance"]
median = np.median(x)
mad = np.median(np.absolute(x- np.median(x)))
lower_threshold = median - 3*mad
upper_threshold = median + 3*mad

print("median:", median)
print("mad:", mad)
print("lower_threshold:", lower_threshold)
print("upper_threshold:", upper_threshold)

# remove outlier
outlier_index = gsr.query("Resistance < @lower_threshold or Resistance > @upper_threshold").index
gsr.drop(outlier_index, inplace = True)

# make timestamp readable
gsr['readableTimestamp'] = pd.to_datetime(gsr['timestamp'], unit='ms')

fig = px.scatter(gsr, x="readableTimestamp", y="Resistance")
fig.show()

median: 5838.0
mad: 335.0
lower_threshold: 4833.0
upper_threshold: 6843.0


### Q4

Please (1) apply mean and interpolate imputation method for 'K_EmoPhone_250L.csv' in your output folder and (2) save the output as 'KEmoPhone_250L_imputation.csv'. (If you have finished HW#1, then you have 'KEmoPhone_250L.csv' in your output folder.) **(0.75 points)**

**Only use first 500 rows**

*   Apply mean imputation method for numeric columns of 'K_EmoPhone_250L.csv' in your output folder. (0.25 points)
*   Apply interpolate imputation method for numeric columns of 'K_EmoPhone_250L.csv' in your output folder. (0.25 points)
*   Save the output as `K_EmoPhone_250L_imputation_mean.csv` and `K_EmoPhone_250L_imputation_interpolate.csv`. (0.25 points)

In [108]:
# Write your code here
KEmoPhone_250L = pd.read_csv(os.path.join(class_dir, 'output', 'KEmoPhone_250L.csv'))

# use the first 500 rows
KEmoPhone_250L = KEmoPhone_250L[:500]
#KEmoPhone_250L.head()

KEmoPhone_250L_mean_imputed = impute_mean(KEmoPhone_250L.copy(), col=["x","y", "z", "temperature", "bpm"] )
KEmoPhone_250L_interpolate_imputed = impute_interpolate(KEmoPhone_250L.copy(), col=["x","y", "z", "temperature", "bpm"] )

KEmoPhone_250L_mean_imputed.to_csv(os.path.join(class_dir, 'output', 'K_EmoPhone_250L_imputation_mean.csv'))
KEmoPhone_250L_interpolate_imputed.to_csv(os.path.join(class_dir, 'output', 'K_EmoPhone_250L_imputation_interpolate.csv'))

display(KEmoPhone_250L_mean_imputed.head(),KEmoPhone_250L_interpolate_imputed.head())


<ipython-input-91-87833c02e50a>:5: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



,readableTimestamp,bpm,x,y,z,temperature,motion
0,2019-05-13 01:19:11.500,76.976,0.038330,0.537354,0.871094,34.005999,STILL
1,2019-05-13 01:19:11.750,76.976,0.036865,0.530762,0.872559,33.799999,STILL
2,2019-05-13 01:19:12.000,76.976,0.035645,0.534546,0.871094,34.005999,STILL
3,2019-05-13 01:19:12.250,78.000,0.035645,0.534424,0.871338,34.005999,STILL
4,2019-05-13 01:19:12.500,76.976,0.037598,0.534302,0.872559,34.005999,STILL


,readableTimestamp,bpm,x,y,z,temperature,motion
0,2019-05-13 01:19:11.500,78.0,0.038330,0.537354,0.871094,33.799999,STILL
1,2019-05-13 01:19:11.750,78.0,0.036865,0.530762,0.872559,33.799999,STILL
2,2019-05-13 01:19:12.000,78.0,0.035645,0.534546,0.871094,33.801379,STILL
3,2019-05-13 01:19:12.250,78.0,0.035645,0.534424,0.871338,33.802758,STILL
4,2019-05-13 01:19:12.500,78.0,0.037598,0.534302,0.872559,33.804137,STILL
